In [ ]:
import connection
import summarization
import embedding
import data_visualization
import clusterization
from openai import OpenAI

### Text encoding

In [2]:
TEXT_ENCODING = "cl100k_base" # Encodings specify how text is converted into tokens
MAX_TOKENS = 4096
MIN_NUMBER_OF_MESSAGES = 100 # Número mínimo de mensagens que definimos pra fazer a sumarização com base na CDF que fizemos pra quantidade de mensagens/dia grupo.

### Messages retrieval from all groups

In [6]:
groups = connection.get_groups()

### Separate messages for each group from the defined Timestamp (2023-01-02 to 2023-01-19)

In [ ]:
connection.get_separated_messages(groups)

### Models Summarization
As células abaixo são referentes a sumarizações que foram testadas utilizando diversos modelos disponíveis atualmente.

#### Summarization from _NVIDIA_

In [3]:
file_dir = 'testes_llms/timestamp_day_2_to_18_summaries/summaries_all_groups_gpt4o_mini_min_messages_100.txt'
with open(file_dir, 'r') as f:
    summaries = eval(f.read())

In [9]:
#Chave antiga: 
nvidia_key = "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"
#nvidia_key = "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"
model_nvidia = "nvidia/nemotron-4-340b-instruct"

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = nvidia_key
)

#### Sumarization from _gpt-4o-mini_

In [13]:
OPEN_AI_API_KEY = "sk-proj-cz4tH6bRNpmeJ6XUWKRdT3BlbkFJ3LENTNjrqEVxJVsy2D4s"
openai_model = "gpt-4o-mini"
APIclient = OpenAI(api_key = OPEN_AI_API_KEY)

In [ ]:
summaries = summarization.get_summaries_for_groups(groups, TEXT_ENCODING, openai_model, MAX_TOKENS, MIN_NUMBER_OF_MESSAGES, APIclient)

#### Summarization from _Maritalk_

In [5]:
import maritalk
API_KEY_MARITALK = "114784108350362647579$2fc4910d7c5aab63"

model_maritalk = maritalk.MariTalk(
    key = API_KEY_MARITALK,
    model = "sabia-3"
)

In [ ]:
summaries = summarization.get_summaries_for_groups_maritalk(groups, TEXT_ENCODING, model_maritalk, MAX_TOKENS)

#### Summarization from _Llama3_

In [6]:
from groq import Groq
API_KEY_LLAMA3 = "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"
LLAMA3_MODEL = "llama3-groq-70b-8192-tool-use-preview"
APIclient_llama3 = Groq(api_key = API_KEY_LLAMA3)

In [ ]:
summarization_from_llama = summarization.get_summaries_for_groups(groups, TEXT_ENCODING, LLAMA3_MODEL, MAX_TOKENS, APIclient_llama3)

### Embedding

In [9]:
from transformers import AutoModel
from transformers import AutoTokenizer

In [ ]:
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case = True)
embeddings = embedding.get_embeddings(summaries, tokenizer, model)

In [4]:
embeddings = embedding.get_embedding_saved('embedding_gpt-4o_day2_to_18_min_messages_100pk1')

In [7]:
labels = embedding.get_labels(summaries)
date_labels = embedding.get_date_labels(groups, TEXT_ENCODING, MAX_TOKENS, MIN_NUMBER_OF_MESSAGES)

### Dimensionality Reduction

#### t-SNE

In [13]:
low_dim_embeddings = embedding.tsne_reduce_dim(embeddings, summaries)
print("Embedding reduzido para 2 dimensões:", low_dim_embeddings.shape)

Dimensão do embedding original: 1024
Dimensão do embedding depois de aplicar o t-SNE: 2
Embedding reduzido para 2 dimensões: (2520, 2)


#### UMAP

In [152]:
from umap import UMAP
import numpy as np

def umap_reduce_dim(embeddings, n_dimensions):
    umap_2d = UMAP(n_components = n_dimensions, spread = 0.8, min_dist = 0.2, n_neighbors = 10)

    low_dim_embeddings = umap_2d.fit_transform(embeddings)

    print(f"Dimensão do embedding original: {embeddings.shape[1]}")
    print(f"Dimensão do embedding depois de aplicar o UMAP: {np.array(low_dim_embeddings).ndim}")

    return low_dim_embeddings

In [153]:
low_dim_embeddings_umap = umap_reduce_dim(embeddings, 2)
print("Embedding reduzido para 2 dimensões: ", low_dim_embeddings_umap.shape)

Dimensão do embedding original: 1024
Dimensão do embedding depois de aplicar o UMAP: 2
Embedding reduzido para 2 dimensões:  (640, 2)


In [154]:
high_dim_embeddings_umap = embedding.umap_reduce_dim(embeddings, 10)
print("Embedding reduzido para 10 dimensões: ", high_dim_embeddings_umap.shape)

Dimensão do embedding original: 1024
Dimensão do embedding depois de aplicar o UMAP: 2
Embedding reduzido para 10 dimensões:  (640, 10)


### Groups Trajectories
Nessa seção é calculada a trajetória dos grupos no período analisado ('2023-01-06 a 2023-01-18).  
A partir do cálculo da trajetória, é também calculado a _média_ dessa trajetória e o _desvio padrão_ da trajetória dos grupos.

In [ ]:
trajectory_df = embedding.get_df_for_trajectory(high_dim_embeddings_umap, labels, date_labels)
groups_trajectories = embedding.get_all_groups_trajectories(trajectory_df)
trajectory_scatter_plot = data_visualization.trajectory_plot(groups_trajectories)
trajectory_scatter_plot.show()

É plotado também o Embedding dos sumários para comparação com a trajetória dos grupos.

In [ ]:
df_for_plot = data_visualization.get_df_for_plot(low_dim_embeddings_umap, labels, date_labels)
fig, df = data_visualization.embedding_scatter_plot(df_for_plot)
fig.show()

### Função de distribuição acumulada

Abaixo, é mostrado o CDF da quantidade de mensagens por dia de cada grupo, com o objetivo de verificar, se selecionarmos uma quantidade de mensagens aleatória X, qual a probabilidade dessa quantidade (X) ser menor ou igual a determinado x.

- Exemplo: Se selecionarmos aleatoriamente uma quantidade de mensagens, qual a probabilidade de que contenha até 100 mensagens?  
 De acordo com o gráfico, aproximadamente 80.6%.

In [19]:
df_for_cumulative_distribution_function = data_visualization.get_df_for_distribution_function(groups)
messages_cumulative_fig, active_users_cumulative_fig = data_visualization.cumulative_distribution_function(df_for_cumulative_distribution_function)
messages_cumulative_fig.show()
active_users_cumulative_fig.show()

In [41]:
fig1, fig2 = data_visualization.cumulative_distribution_function(df_for_cumulative_distribution_function)

In [42]:
fig1.show()

### Clusterização

#### K-Means
Métrica para avaliar a quantidade de Clusters:
- Coeficiente de Silhueta

Quanto maior essa métrica, mais bem definidos são os clusters do modelo.
Consideramos duas pontuações:  
_a_: Distância média entre uma amostra e todos os outros pontos da mesma classe.  
_b_: Distância média entre uma amostra e todos os outros pontos no cluster mais próximo.

Ou seja, avalia tanto a distância **intraclusters** quanto a distância **interclusters**.
Para uma amostra, temos:

$$s = \frac{b - a} {max(a, b)}$$
Para o conjunto de amostras, o coeficiente é a média dos valores que obtemos para cada amostra.

In [155]:
df, best_k = clusterization.find_best_k(high_dim_embeddings_umap, 10)
k_means = clusterization.k_means_clustering(df['num_clusters'][1], high_dim_embeddings_umap)

In [156]:
dff = data_visualization.get_df_for_plot(low_dim_embeddings_umap, labels, date_labels, k_means.labels_)
fig_kmeans, dfff = data_visualization.embedding_scatter_plot(dff)
fig_kmeans.show()

#### HDBScan

In [170]:
hdbscan_cluster, hdbscan_labels = clusterization.hdbscan_clustering(high_dim_embeddings_umap, 30)

In [ ]:
df_hdbscan, best_hdbscan_cluster = clusterization.get_silhouette_score_hdbscan(high_dim_embeddings_umap, 10)
df_hdbscan

In [171]:
dff = data_visualization.get_df_for_plot(low_dim_embeddings_umap, labels, date_labels, hdbscan_labels)
fig_hdbscan, dfff = data_visualization.embedding_scatter_plot(dff)
fig_hdbscan.show()

### Dash 

In [ ]:
app = data_visualization.dash_app(fig_hdbscan, fig_kmeans, trajectory_scatter_plot, dff)
data_visualization.dash_callback(app, dff)
data_visualization.run_server(app)